<div class="elfjS" data-track-load="description_content"><p>A <a href="https://en.wikipedia.org/wiki/Trie" target="_blank"><strong>trie</strong></a> (pronounced as "try") or <strong>prefix tree</strong> is a tree data structure used to efficiently store and retrieve keys in a dataset of strings. There are various applications of this data structure, such as autocomplete and spellchecker.</p>

<p>Implement the Trie class:</p>

<ul>
	<li><code>Trie()</code> Initializes the trie object.</li>
	<li><code>void insert(String word)</code> Inserts the string <code>word</code> into the trie.</li>
	<li><code>boolean search(String word)</code> Returns <code>true</code> if the string <code>word</code> is in the trie (i.e., was inserted before), and <code>false</code> otherwise.</li>
	<li><code>boolean startsWith(String prefix)</code> Returns <code>true</code> if there is a previously inserted string <code>word</code> that has the prefix <code>prefix</code>, and <code>false</code> otherwise.</li>
</ul>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre><strong>Input</strong>
["Trie", "insert", "search", "search", "startsWith", "insert", "search"]
[[], ["apple"], ["apple"], ["app"], ["app"], ["app"], ["app"]]
<strong>Output</strong>
[null, null, true, false, true, null, true]

<strong>Explanation</strong>
Trie trie = new Trie();
trie.insert("apple");
trie.search("apple");   // return True
trie.search("app");     // return False
trie.startsWith("app"); // return True
trie.insert("app");
trie.search("app");     // return True
</pre>

<p>&nbsp;</p>
<p><strong>Constraints:</strong></p>

<ul>
	<li><code>1 &lt;= word.length, prefix.length &lt;= 2000</code></li>
	<li><code>word</code> and <code>prefix</code> consist only of lowercase English letters.</li>
	<li>At most <code>3 * 10<sup>4</sup></code> calls <strong>in total</strong> will be made to <code>insert</code>, <code>search</code>, and <code>startsWith</code>.</li>
</ul>
</div>

In [ ]:
from typing import *

In [ ]:
class Node:
    def __init__(self, value: str, terminal: bool):
        """
        Initializes a node structure that a Trie tree is composed of.
        :param value:       The character value of the node (should not be a string with more than one character).
        :param terminal:    If true, this character marks the end of a word.
        """
        self.value = value
        self.terminal = terminal
        # this is a bit mask denoting the presence of a child
        self.children_characters = 0x0000
        # this hold the children
        self.children_nodes = [None] * 26
        # converts lower-case letters to an index in the range [0, 25]
        self.get_idx = lambda x : ord(x) - ord('a')

    def hasChild(self, query: str):
        """
        Returns bool indicating if the node has some child. The query should be a string with a single character.
        :param query: Character to search for among the children.
        :return:      Bool indicating if the node has the child.
        """
        hq = self.get_idx(query)
        return ((0x0001 << hq) & self.children_characters) > 0

    def insertChild(self, query: str, terminal: bool):
        """
        Inserts a character as a child of the current node.
        :param query:
        :param terminal:
        :return:
        """
        hq = self.get_idx(query)
        self.children_characters |= (0x0001) << hq
        new_node = Node(query[0], terminal)
        self.children_nodes[hq] = new_node
        return new_node

    def getChild(self, query: str):
        """
        Returns the child of the current node.
        :param query: Child to return.
        :return:      The returned child node.
        """
        hq = self.get_idx(query)
        return self.children_nodes[hq]

class Trie:

    def __init__(self):
        # initialize root of the Trie structure
        self.root: Node = Node("_", True)

    def insert(self, word: str) -> None:
        curr_node = self.root
        for i in range(len(word)):
            # the node associated with the last character in the word should
            # be marked as terminal
            isTerminal = i == len(word) - 1
            if not curr_node.hasChild(word[i]):
                # child does not exist, we must create it
                curr_node = curr_node.insertChild(word[i], isTerminal)
            else:
                # child does exist, retrieve it
                curr_node = curr_node.getChild(word[i])
                # We should appropriately update the node's terminal attribute. If the node is
                # marked as terminal, but we are not at the last character of our current word,
                # we should not set it to False otherwise we neglect the presence of some other
                # word in our tree.
                curr_node.terminal |= isTerminal

    def _searchPrefix(self, prefix: str) -> Optional[Node]:
        """
        Returns the node associated with the last character in the prefix if its exist.
        :param prefix:  Prefix string which describes how to traverse the tree.
        :return:        Node of the last character in the prefix, otherwise None.
        """
        curr_node = self.root
        for i in range(len(prefix)):
            char = prefix[i]
            if not curr_node.hasChild(char):
                return None
            curr_node = curr_node.getChild(char)
        return curr_node

    def search(self, word: str) -> bool:
        """
        Search of the complete word exists in the tree.
        :param word:    Complete word to search for.
        :return:        Bool if word exists.
        """
        curr_node = self._searchPrefix(word)
        return curr_node is not None and curr_node.terminal


    def startsWith(self, prefix: str) -> bool:
        """
        Search if the prefix exists in the tree.
        :param prefix:  Prefix string to search for.
        :return:        Bool if prefix exists.
        """
        return self._searchPrefix(prefix) is not None

# Your Trie object will be instantiated and called as such:
# obj = Trie()
# obj.insert(word)
# param_2 = obj.search(word)
# param_3 = obj.startsWith(prefix)